In [48]:
%gui qt

from OCC.Display.SimpleGui import init_display
from OCC.Core.gp import gp_Pnt, gp_Dir, gp_Pln, gp_Vec
from OCC.Core.BRepAlgoAPI import BRepAlgoAPI_Section
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_EDGE
from OCC.Core.TopoDS import topods
from OCC.Core.GProp import GProp_GProps
from OCC.Core.BRepGProp import brepgprop
from OCC.Core.BRepBuilderAPI import BRepBuilderAPI_MakeEdge, BRepBuilderAPI_MakeWire, BRepBuilderAPI_MakeFace
import numpy as np

# Start display without blocking
display, start_display, add_menu, add_function_to_menu = init_display()

display.EraseAll()
display.View_Iso()
display.FitAll()

pyside6 backend - Qt version 6.8.3


In [49]:
# Load the STEP and extract 'solid'
from OCC.Extend.DataExchange import read_step_file
# shape = read_step_file("../data/0444-1 ANGLED.step")
# shape = read_step_file("../data/ncTest.step")
shape = read_step_file("../data/TestEA.step")
# shape = read_step_file("../data/TestUEA.step")
# shape = read_step_file("../data/TestPFC.step")
from OCC.Core.TopExp import TopExp_Explorer
from OCC.Core.TopAbs import TopAbs_SOLID
exp = TopExp_Explorer(shape, TopAbs_SOLID)
solid = exp.Current()

In [50]:
def compute_obb_and_local_axes(solid):
    from OCC.Core.Bnd import Bnd_OBB
    from OCC.Core.BRepBndLib import brepbndlib
    from OCC.Core.gp import gp_Vec

    # Compute OBB
    obb = Bnd_OBB()
    brepbndlib.AddOBB(solid, obb, True, True, False)

    # Get raw data
    center = obb.Center()
    axes = [obb.XDirection(), obb.YDirection(), obb.ZDirection()]
    half_extents = [obb.XHSize(), obb.YHSize(), obb.ZHSize()]

    # Sort half-extents to identify logical axes
    sorted_indices = np.argsort(half_extents)
    web_idx, flange_idx, length_idx = sorted_indices

    # Assign named axes + extents
    xaxis = gp_Vec(axes[web_idx])
    yaxis = gp_Vec(axes[flange_idx])
    zaxis = gp_Vec(axes[length_idx])

    he_X = half_extents[web_idx]
    he_Y = half_extents[flange_idx]
    he_Z = half_extents[length_idx]

    return {
        'center': center,
        'xaxis': xaxis,
        'yaxis': yaxis,
        'zaxis': zaxis,
        'he_X': he_X,
        'he_Y': he_Y,
        'he_Z': he_Z,
        'axes': (xaxis, yaxis, zaxis),
        'half_extents': (he_X, he_Y, he_Z)
    }


In [51]:
def draw_plane(display, origin, normal, size=50, color="MAGENTA"):
    """
    Draws a square plane centered at `origin`, normal to `normal` axis vector.
    
    Parameters:
        display -- OCC display
        origin  -- gp_Pnt (plane center)
        normal  -- gp_Vec (plane normal)
        size    -- length of plane sides
        color   -- display color
    """
    from OCC.Core.gp import gp_Vec, gp_Pnt
    import numpy as np

    # Convert normal to numpy
    n = np.array([normal.X(), normal.Y(), normal.Z()])
    n = n / np.linalg.norm(n)

    # Choose a reference vector not colinear with `n`
    ref = np.array([0, 0, 1])
    if np.abs(np.dot(n, ref)) > 0.99:
        ref = np.array([1, 0, 0])

    # Compute local u, v plane axes
    u = np.cross(n, ref); u = u / np.linalg.norm(u)
    v = np.cross(n, u);   v = v / np.linalg.norm(v)

    # Convert origin to numpy
    o = np.array([origin.X(), origin.Y(), origin.Z()])
    s = size / 2.0

    # Create four corner points
    pts = [
        gp_Pnt(*(o + s * ( u + v))),
        gp_Pnt(*(o + s * (-u + v))),
        gp_Pnt(*(o + s * (-u - v))),
        gp_Pnt(*(o + s * ( u - v))),
    ]

    # Create edges between corners
    for i in range(4):
        e = BRepBuilderAPI_MakeEdge(pts[i], pts[(i+1)%4]).Edge()
        display.DisplayShape(e, color=color, update=False)


In [52]:
def cut_section_at_start(solid, center, zaxis, he_Z, display=None, show_plane=True):
    """
    Cuts a cross-section at the start of the beam (negative Z direction from center).

    Parameters:
        solid   -- The STEP solid (TopoDS_Shape)
        center  -- gp_Pnt of the OBB center
        zaxis   -- gp_Vec (local length axis)
        he_Z    -- Half-length (float)
        display -- OCC Display instance (optional, for visualisation)
        show_plane -- If True, shows cutting plane

    Returns:
        face  -- The section face (TopoDS_Face)
        wire  -- The section wire (TopoDS_Wire)
        area  -- Section area (float)
        origin -- gp_Pnt of the cutting plane origin
    """
    # Compute start origin: center - he_Z * zaxis
    origin_vec = np.array([center.X(), center.Y(), center.Z()]) - he_Z * np.array([zaxis.X(), zaxis.Y(), zaxis.Z()])
    origin = gp_Pnt(*origin_vec)

    # Plane normal is aligned with zaxis (beam length direction)
    plane = gp_Pln(origin, gp_Dir(zaxis))

    # Perform section
    sec = BRepAlgoAPI_Section(solid, plane)
    sec.ComputePCurveOn1(True)
    sec.Approximation(True)
    sec.Build()

    # Build wire
    exp_e = TopExp_Explorer(sec.Shape(), TopAbs_EDGE)
    wb = BRepBuilderAPI_MakeWire()
    while exp_e.More():
        wb.Add(topods.Edge(exp_e.Current()))
        exp_e.Next()
    wire = wb.Wire()

    # Create face and compute area
    face = BRepBuilderAPI_MakeFace(wire).Face()
    gp2 = GProp_GProps()
    brepgprop.SurfaceProperties(face, gp2)
    area = gp2.Mass()

    # Optionally show plane
    if display and show_plane:
        draw_plane(display, origin, zaxis, size=50, color="RED")
        display.DisplayShape(origin, color="RED", update=False)
        display.DisplayShape(wire, color='CYAN', update=True)

    return face, wire, area, origin


In [53]:
def draw_obb(display, center, xaxis, yaxis, zaxis, he_X, he_Y, he_Z, color="WHITE"):
    # Convert center to numpy array
    c = np.array([center.X(), center.Y(), center.Z()])

    # Convert each axis to numpy vector
    xvec = np.array([xaxis.X(), xaxis.Y(), xaxis.Z()])
    yvec = np.array([yaxis.X(), yaxis.Y(), yaxis.Z()])
    zvec = np.array([zaxis.X(), zaxis.Y(), zaxis.Z()])

    # Generate 8 corner points
    corners = []
    for dx in (-1, 1):
        for dy in (-1, 1):
            for dz in (-1, 1):
                offset = dx * he_X * xvec + dy * he_Y * yvec + dz * he_Z * zvec
                pt = gp_Pnt(*(c + offset))
                corners.append(pt)

    # Define OBB wireframe edges (12 edges)
    edges = [
        (0, 1), (0, 2), (0, 4),
        (1, 3), (1, 5),
        (2, 3), (2, 6),
        (3, 7),
        (4, 5), (4, 6),
        (5, 7),
        (6, 7)
    ]

    # Draw edges
    for i, j in edges:
        edge = BRepBuilderAPI_MakeEdge(corners[i], corners[j]).Edge()
        display.DisplayShape(edge, color=color, update=False)


In [54]:
# Display solid
display.DisplayShape(solid, update=True)

# Compute OBB
obb_data = compute_obb_and_local_axes(solid)

center     = obb_data['center']
xaxis      = obb_data['xaxis']
yaxis      = obb_data['yaxis']
zaxis      = obb_data['zaxis']
he_X       = obb_data['he_X']
he_Y       = obb_data['he_Y']
he_Z       = obb_data['he_Z']

print("OBB center:", center.X(), center.Y(), center.Z())
print("Half-extents:", he_X, he_Y, he_Z)
print(f"Axis X: {xaxis}, Y: {yaxis}, Z: {zaxis}")

# Draw bounding box
draw_obb(display,
         center=obb_data['center'],
         xaxis=obb_data['xaxis'],
         yaxis=obb_data['yaxis'],
         zaxis=obb_data['zaxis'],
         he_X=obb_data['he_X'],
         he_Y=obb_data['he_Y'],
         he_Z=obb_data['he_Z'],
         color="YELLOW")

# Draw end section plane
face, wire, area, start_origin = cut_section_at_start(
    solid,
    center=obb_data['center'],
    zaxis=obb_data['zaxis'],
    he_Z=obb_data['he_Z'],
    display=display
)

print(f"Section area at start: {area:.2f} mm²")


display.FitAll()

OBB center: 10.939112487100099 10.939112487100099 500.0
Half-extents: 25.0 25.0 500.0
Axis X: <class 'gp_Vec'>, Y: <class 'gp_Vec'>, Z: <class 'gp_Vec'>
Many colors for color name CYAN, using first.
Section area at start: 480.26 mm²
